# Sentinel-2 on the Planetary Computer

This notebook explores Sentinel-2 data on Microsoft's Planetary Computer using:

- [Planetary Computer STAC API](https://planetarycomputer.microsoft.com/api/stac/v1), catalog of public data
- [Planetary Computer Hub](https://planetarycomputer.microsoft.com/) for running Jupyter Notebooks in the cloud
- [pystac-client](https://pystac-client.readthedocs.io/) for searching and access data
- [OpenDataCube](https://www.opendatacube.org/) and [odc-stac](https://odc-stac.readthedocs.io/) for loading STAC assets and representing geospatial data as XArrays
- [XArray](http://xarray.pydata.org/en/stable/), [pandas](https://pandas.pydata.org/) and [geopandas](https://geopandas.org/) for manipulating data
- [Dask](https://dask.org/) for performing parallel, distributed computing
- [hvplot](https://hvplot.holoviz.org/) for visualization

Shown will be how find data for an area of interest, explore the resulting metadata, perform calculations, and visualize the results.

Created by [Element 84](http://element84.com/)

In [ ]:
# initial imports and reusable functions

import holoviews as hv
hv.extension('bokeh')

from copy import deepcopy
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
from shapely.geometry import shape

# create a function for later reuse
def plot_polygons(data, *args, **kwargs):
    return data.hvplot.paths(*args, geo=True, tiles='OSM', xaxis=None, yaxis=None,
                             frame_width=600, frame_height=600,
                             line_width=3, **kwargs)

# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i['geometry'] = shape(_i['geometry'])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ['properties.datetime', 'properties.created', 'properties.updated']:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index('properties.datetime', inplace=True)
    return gdf

# set pystac_client logger to DEBUG to see API calls
import logging
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.INFO)

# Search for data

Use pystac-client to find data in the Planetary Computer STAC API.

In [ ]:
# Open the Planetary Computer STAC API

from pystac_client import Client
URL = 'https://planetarycomputer.microsoft.com/api/stac/v1'
cat = Client.open(URL)
cat

Fetch the collection of interest: Sentinel-2, Level 2 Surface Reflectance and print the assets that are available.

In [ ]:
collection = cat.get_collection('sentinel-2-l2a')

pd.DataFrame.from_dict(collection.to_dict()['item_assets'], orient='index')

Change the AOI, search parameters here

In [ ]:
import geopandas as gpd
import json

aoi = gpd.read_file('../aois/bear-fire.geojson')
geom = aoi['geometry'][0] # < shapely geometry object

# limit sets the # of items per page so we can see multiple pages getting fetched
search = cat.search(
    collections = ["sentinel-2-l2a"],
    intersects = geom,
    datetime = "2019-10-01/2019-10-31",
    query = ["eo:cloud_cover<25"],
    limit = 100
)

# Use GeoPandas to view footprints

The cell below fetches all the STAC Items, then creates a GeoDataFrame for visualizing the footprints.

In [ ]:
# Get all items as a dictionary
items_dict = search.get_all_items_as_dict()['features']

# Create GeoDataFrame from Items
items_gdf = items_to_geodataframe(items_dict)

print(f"{len(items_dict)} items found")

items_gdf.head()

In [ ]:
plot_polygons(aoi) * items_gdf.hvplot.paths(geo=True)

# OpenDataCube

Now we'll turn the set of scenes into a virtual datacube. None of the data will actually be read yet.

The configuration string (`cfg`) is for providing additional info not currently available in the STAC Items, but will be in the future.

In [ ]:
import yaml

cfg = """---
sentinel-2-l2a:
  assets:
    '*':
      data_type: uint16
      nodata: 0
      unit: '1'
"""
cfg = yaml.load(cfg, Loader=yaml.CSafeLoader)

Here we load as a DataCube. A PySTAC ItemCollection is created from the found STAC Items, and we specify various parameters, such as bands of interest and chunk size. We are requesting to only load pixels within a bounding box of the requested geometry (`bbox=geom.bounds`).

In [ ]:
%%time

from odc.stac import stac_load
import planetary_computer as pc

# Create PySTAC ItemCollection
item_collection = pystac.ItemCollection(items_dict)

dc = stac_load(item_collection,
               measurements=['B02', 'B03', 'B04', 'B08'],
               chunks={"x": 2048, "y": 2048},
               bbox=geom.bounds,
               stac_cfg=cfg,
               patch_url=pc.sign
)
dc

# Calculations

We will create an RGBA datacube representation (`nodata` values have `alpha=0`), and generate an NDVI datacube.

In [ ]:
from odc.algo import to_rgba

RGB = ['B04', 'B03', 'B02']
vis = to_rgba(dc, clamp=(1, 3000), bands=RGB)
vis

In [ ]:
ndvi = ((dc['B08'] - dc['B04']) / (dc['B08'] + dc['B04'])).clip(0, 1).rename("ndvi")
ndvi

# Start Dask Client

Start a local Dask cluster.

In [ ]:
from dask.distributed import Client
client = Client()
client

# Compute

Now, we kick off our Dask computation by using the Dask persist function, which will keep the data in memory on the cluster for faster access later.

The Dask `compute` function is used when we actually want the data, such as displaying it.

In [ ]:
%%time
from dask.distributed import wait

ndvi, vis = client.persist([ndvi, vis])
_ = wait([ndvi, vis])

In [ ]:
%%time
vis_ = vis.compute()
vis_.plot.imshow(col='time', rgb='band', col_wrap=5, robust=True)

In [ ]:
import hvplot.xarray

hvplot_kwargs = {
    "frame_width": 800,
    "xaxis": None,
    "yaxis": None,
    "widget_location": "bottom",
    "aspect": len(vis.x)/len(vis.y)
}

vis_.hvplot.rgb('x', 'y', bands='band', groupby='time', **hvplot_kwargs)

In [ ]:
ndvi_ = ndvi.compute()
ndvi_.hvplot('x', 'y', groupby='time', **hvplot_kwargs)

Create an animated GIF of NDVI over time using `geogif` with the fetched results.

In [ ]:
%%time
from geogif import gif, dgif

gif(ndvi_, fps=5, cmap='YlGn')

In [ ]:
%%time
ndvi_mean = ndvi.mean(dim=['x', 'y']).compute()
ndvi_mean.hvplot()

# Shutdown cluster

Shut down the cluster.

In [ ]:
if "cluster" in locals():
    cluster.close()